# Force fields


## Bonded and non-bonded interactions

Even with all the approximations and optimizations of the last hundred years, first principles quantum chemical calculations are still very resource demanding, even for quite small systems. The upper limit for a geometry optimization is around a thousand atoms and a dynamics simulation is only possible for systems of a few hundred atoms, and even then only for very short time scales. This means that unless new approximations are introduced, many systems that are of interest will be far beyond the capabilities of today's computers. A small piece of protein from the human body can consist of several thousand atoms, and even very small systems can increase by hundreds of atoms when a solvent is added. A popular way of dealing with this problem when simulating the motion of atoms is to use molecular mechanics (MM).

In electronic structure theory, the Born--Oppenheimer approximation is utilized to separate the Schr\"o{}dinger equation into electronic and nuclear parts that can be solved separately. The electrons are approximated as responding instantly to any changes in the nuclear coordinates, allowing the electronic distribution to be found around stationary nuclei, after which the problem is solved for the nuclei moving in the effective potential generated by the electrons. MM takes this one step further and combines nuclei and electrons into a single unit, unsurprisingly referred to as atoms. The potential energy of the system is then approximated as a simple analytical function of the atomic coordinates. This potential energy function is split into contributions from different kinds of interactions involving two or more atoms, such as bond stretching and angle bending. The specific form of the potential energy function as well as the parameters used in it are know as a force field. The included interactions and the functional form that they take can differ depending on the requirements of the force field, with some focused on describing specific groups of molecules well and some sacrificing accuracy for computational efficiency. A simple energy function can look like
%
\begin{equation*}
E = {E_\mathrm{s}}  + {E_\theta} + {E_\omega} + {E_\mathrm{vdw}} + {E_\mathrm{el}}
\end{equation*}
%
where each term is a sum, adding up all the contribution from a certain kind of bonded or non-bonded interaction in the system.

The potential energy function of the system has as its variables the atomic coordinates, but it also depends parametrically on how the atoms are bonded and the types of the involved atoms. In MM, atom type does not just mean which element the atom belongs to, but also its chemical environment. This concept of atom types contains one of the main ideas behind MM force fields, which is that molecular structures that exist in similar environments behave in a similar manner. As an example, the carbon--carbon bond found in ethane is very similar to the two found in propane. The bond lengths are similar and the changes in potential energy when the bonds are stretched or contracted are almost the same. The parameters used in the potential energy function describing these three bonds can therefore safely be approximated as being the same. This transferability is one of the main strengths of using force fields, as parameters developed for a certain structure in one system can be reused for similar structures in other systems. For example, the parameters needed to describe butane are all that is required to describe all linear and branched alkanes in a simple force field.

While the carbon--carbon bonds in ethane and propane are similar enough for the parameters to be transferable, the same cannot be said for the carbon--carbon triple bond in acetylene, where the bond length is much shorter and the bond strength is significantly higher. Using the same parameters for all these bonds would lead to a poor description, which indicates that a finer division of atom types is needed than to just have one for each element. How fine this division is depends on what is desired from the force field. Creating accurate parameters can be a difficult and time-consuming process, with fittings made either to experimental data or higher level quantum mechanical calculations. A larger number of atom types makes it possible to give a more nuanced description of the interactions in the system, but it also strips away the transferability advantages. With more atom types the number of parameters quickly increases, and with them the likelihood that some of them will have to be created when studying a new system. 


## Force-field terms

The following section describes the most common force-field interactions, using the CHARMM \cite{brooks1983} and MM3 \cite{allinger1989} force fields as examples. The two main categories of interaction terms are bonded and non-bonded. Bonded interactions exist for atoms that are part of the same molecule, the structure of which is defined at the start of the simulation by the user. There are also reactive force fields, such as ReaxFF \cite{vanduin2001}, where bonds can be formed and broken dynamically. Non-bonded terms appear between atoms in different molecules as well as between atoms in the same molecule, provided that they are separated by at least a certain number of bonds---usually three. For atom pairs connected by fewer bonds, the non-bonded interactions are incorporated into the bonded interactions. 


### Bond stretching
Stretching or contracting a bond from its equilibrium position causes the energy of the system to rise. This change in energy is approximated by the bond stretch terms, usually represented by a simple harmonic oscillator of the form
%
\begin{equation*}
\label{eq:bond}
{E_\mathrm{s}} = \frac{k_\text{s}}{2} (l - l_0)^2
\end{equation*}
%
where $k_\text{s}$ is a force constant, defining the strength of the bond, $l$ is the distance between the two bonded atoms and $l_0$ is the equilibrium distance for the interaction. Both $k_\text{s}$ and $l_0$ are constants specific to the combination of atom types involved in the interaction. It should be noted that while $l_0$ is the bond length that minimizes Equation \eqref{eq:bond}, it is not necessarily the equilibrium distance found in a molecule. As the two bonded atoms are most likely also involved in several other interactions, the molecular equilibrium distance will be the one that minimizes the full potential.

Generally, contracting a bond produces a steeper rise in energy than stretching it. This is accounted for in some force fields, such as MM3, which introduce anharmonicity into the bond energy expression by including higher order terms based on the Morse potential.[cite morse1929] The MM3 bond stretching term has the form
%
\begin{equation*}
\label{eq:bondMM3}
{E_\mathrm{s}} = \frac{k_\text{s}}{2} (l - l_0)^2 \left( 1 - \alpha (l - l_0) + \alpha \frac{7}{12} (l - l_0)^2 \right)
\end{equation*}
%
where $\alpha$ is derived as 2.55 from the Morse potential, but could be taken as an additional parameter. Figure \ref{fig:methanestretch} shows the change in potential energy when stretching and contracting a hydrogen--carbon bond in methane, calculated using an \emph{ab initio} method as well as Equations \eqref{eq:bond} and \eqref{eq:bondMM3}. While the difference is significant for large distortions, the variations around the equilibrium are relatively small at room temperature, meaning the approximations are quite valid.


```{figure} ../../img/md/MM_stretch.svg
---
width: 300px
name: fig_bondstretch
---
Bond stretching.
```

### Angle bending

Same as for bond stretching, distorting an angle formed between two atoms bound to a common third atom from its equilibrium causes the energy of the system to rise. This too is commonly approximated by a harmonic oscillator of the form
%
\begin{equation*}
{E_\theta} = \frac{k_\theta}{2} (\theta - \theta_0)^2
\end{equation*}
%
where $k_\theta$ is a force constant, $\theta$ is the angle formed by the atoms and $\theta_0$ is the equilibrium angle. The real angle bending interaction is clearly not a harmonic oscillator, as exemplified by the potential energy curve in Figure \ref{fig:waterbend}, showing the change in energy when distorting the H--O--H angle found in water. Compressing the angle causes the two hydrogen atoms to come close together, leading to a steep curve near $\theta = 0^\circ$, while increasing the angle leads to a cusp at $\theta = 180^\circ$ due to symmetry. Again, some force fields account for this anharmonicity with additional terms, such as the MM3 angle bending term:
%
\begin{equation*}
\begin{split}
{E_\theta} = \frac{k_\theta}{2} (\theta - \theta_0)^2 \bigl( 1 &- \alpha (\theta - \theta_0) + \beta (\theta - \theta_0)^2  \\
 & - \gamma (\theta - \theta_0)^3 + \delta (\theta - \theta_0)^4 \bigr)
\end{split}
\end{equation*} 
%
where the constants have been fitted to experimental data but are the same for all bending interactions. This gives a better agreement with the actual angle bending behaviour, as seen in Figure \ref{fig:waterbend}, but again, for lower temperatures the harmonic oscillator is still a valid and computationally efficient approximation.


```{figure} ../../img/md/MM_bend.svg
---
width: 300px
name: fig_MM_bend
---
Angle bending.
```


```{figure} ../../img/md/MM_waterbend_even.svg
---
width: 300px
name: fig_MM_waterbend_even
---
Potential energy curves for distortion of the water angle. The reference \emph{ab initio} energy was calculated at the MP2/cc-pVTZ level of theory while the other curves were derived from the harmonic oscillator used in CHARMM and the anharmonic expression used in MM3.
```


### Torsion
When two atoms are bonded to atoms on opposite sides of a central bond, as for the opposing hydrogen atoms in ethane, then twisting these around the central bond can cause a change in the potential energy. The torsional energy is thus a function of the dihedral angle, $\omega$, formed between the first and fourth atom, while looking through the bond between the second and third atoms. For a visualization of this, see Figure \ref{fig:torsion}. The torsion energy term is usually expressed in terms of a Fourier expansion of the form
%
\begin{equation*}
\label{eq:torsion}
{E_\omega} = k_\omega \left(1 + \cos(n\omega -\delta)\right)
\end{equation*}
%
where $k_\omega$ is a force constant, $n$ is the periodicity and $\delta$ is a phase shift. Typically, several such terms are included for each set of four atoms in order to combine different multiplicities. The MM3 force field always includes terms of multiplicity 1, 2 and 3, with $\delta$ set to zero for each, while the CHARMM force field allows a more varied combination of terms.

Figure \ref{fig:butanetorsion} shows the potential energy curve when twisting butane around the central carbon--carbon bond. The two identical lower peaks represent the conformations when the one hydrogen atom on each side is eclipsed by the carbon atom on the other side, while the main peak comes from the conformation when all atoms are eclipsed by an equivalent atom on the other side. It should be noted that the torsional energy of the force field does not come from a single interaction in this case, but nine. One for the two carbon atoms, four for the carbon atoms meeting the hydrogen atoms on the other side and four for  the combinations of hydrogen atoms on opposite sides. It is easy to see that torsional interactions can quickly become very complicated when there are many atom types involved. This is further complicated by the fact that most force field also include non-bonded interactions between atoms separated by more than two bonds, meaning that the true rotational barrier will be a sum of all torsional and non-bonded interactions. 

The conformation of a molecule is defined by its current dihedral angles, with two identifiable conformations for butane visible as minima in Figure \ref{fig:butanetorsion}: the global minimum at 180 degrees and two identical minima at $\pm70$ degrees. For calculations where both the conformations themselves as well as the transitions between them are important, it is necessary to recreate the full potential energy curve and not just focus on the regions around the minima, as was done for bond stretching and angle bending terms. 


```{figure} ../../img/md/MM_torsion.svg
---
width: 300px
name: fig_MM_torsion
---
Torsion.
```


```{figure} ../../img/md/MM_butanetorsion_even.svg
---
width: 300px
name: fig_MM_butanetorsion_even
---
Potential energy curves for the torsional barrier found when twisting butane around the central bond. The reference \emph{ab initio} energy was calculated at the MP2/cc-pVTZ level of theory while the MM curve was obtained as a least squares fit of Equation \eqref{eq:torsion} with multiplicities 1, 2, 3, 4 and 5.
```


### Electrostatic interaction
The non-bonded force field interactions are divided into electrostatic interactions, caused by static multipoles, and van der Waals interactions, containing everything else. For most force fields, only point charges are used in the electrostatic interactions, resulting in an energy given simply by the Coulomb interaction
%
\begin{equation*}
{E_\mathrm{el}} = \frac{q_i q_j}{4\pi \epsilon_0 r_{ij}},
\end{equation*}
%
where $q_i$ and $q_j$ are the partial charges of the interacting atoms, $r_{ij}$ is the distance between them and \ensuremath{\epsilon_0} is the vacuum permittivity. It is also possible to use higher order multipoles, an example of which can be found in the MM3 force field, where dipole moments are assigned to each bonded pair. This gives the advantage of a more flexible description, as the dipoles depend on the types of both involved atoms, but requires additional parametrization and has difficulty dealing with ionic systems.

\subsubsection{van der Waals interaction}
The van der Waals interaction is typically represented by two terms. One due to exchange interaction, causing a strong repulsion at close distances, and one that is due to London dispersion, resulting in a weak attraction at longer distances. The most commonly used form is the Lennard-Jones potential:[cite jones1924]
%
\begin{equation*}
{E_\mathrm{vdw}} = \epsilon \left[ \left( \frac{r_\text{min}}{r} \right)^{12} - 2 \left( \frac{r_\text{min}}{r} \right)^6 \right]
\end{equation*}

\noindent where $\epsilon$ is the depth of the potential well, usually created as the mean of two parameters specific to the two interacting atom types, $r_\text{min}$ is the position of the bottom of the well and $r$ is the interatomic distance. While it is possible to motivate the use of $r^{-6}$ for the attractive part, the use of $r^{-12}$ for the repulsive part is more pragmatic in origin, having that form simply because it is the square of $r^{-6}$, making its computation highly efficient. It can be argued[cite jensen2006] that an exponential form is more realistic, such as the Buckingham potential[cite buckingham1938] used in MM3:
%
\begin{equation*}
{E_\mathrm{vdw}} = A e^{- B r} - \frac{C}{r^{-6}}
\end{equation*}
%
where $A$, $B$, and $C$ are constants. Calculating this expression is more demanding than the Lennard-Jones (LJ) potential, but produces a repulsive force that is closer to what is found using \emph{ab initio} methods. For very short interatomic distances, however, the Buckingham potential becomes strongly attractive, which can lead to highly unphysical behaviour for systems in unfavourable starting positions.


### Additional terms
Depending on what the force field is intended for, many additional terms can be added to the potential energy function. Cross-terms are a common addition, accounting for more complex couplings between structural properties. Examples of these can be found in the MM3 force field, which contains stretch--bend, tor\-sion--stretch and bend--bend interactions. As an example, the stretch--bend interaction accounts for the fact that compressing an angle formed by three bonded atoms will cause the repulsion between the two end atoms to increase, resulting in an elongation of their bonds to the middle atom. Another example of such structural corrections comes in the form of the out-of-plane term. If three or four atoms are bonded to a central atom with their equilibrium position in a plane, as for e.g.\ formaledehyde, the change in energy caused by a distortion of the central atom out of the plane is hard to model by just angle bending terms, as the individual angle distortions are small. The out-of-plane term instead depends on the angles formed between the plane and the bonds connecting the central and outer atoms. This is shown in Figure \ref{fig:outofplane}, where the $\chi$ is an example of an out-of-plane angle. 


```{figure} ../../img/md/MM_outofplane.svg
---
width: 300px
name: fig_MM_outofplane
---
Out-of-plane bending.
```
